In [1]:
import sys
COVID_dir = '/Users/yiningliu/research/pooled-sampling/COVID-19-pooling' # set this to your directory
sys.path.insert(1, COVID_dir) 

In [2]:
import json
import numpy as np
from test import test_random_M
from membership_matrix import generate_doubly_regular_col, generate_const_col_weight

In [3]:
n = 384
T = 48
num_random_matrices = 25

In [4]:
def get_best_worst(m, k, generate_matrix): 
    const_col_dir = COVID_dir + '/tests/results/' + generate_matrix + '/'
    with open(const_col_dir + 'm%s-k%s-n%s-T%s-numM%s.txt' % (m, k, n, T, num_random_matrices)) as file:
        data = json.load(file)
    best = n
    best_index = 0 
    worst = 0
    worst_index = 0 
    for i in range(len(data)): 
        if data[i]["num_errors"] < best:
            best = data[i]["num_errors"]
            best_index = i
        if data[i]["num_errors"] > worst:
            worst = data[i]["num_errors"]
            worst_index = i
    return data, best_index, worst_index

def get_counts(M):
    inner_prod = M.T @ M
    unique, counts = np.unique(inner_prod, return_counts=True) 
    return dict(zip(unique, counts)) 

def print_inner_product_counts(data, best_index, worst_index):
    M_best = np.array(data[best_index]["membership_matrix"])
    print("good: %s" % get_counts(M_best))
    
    M_bad = np.array(data[worst_index]["membership_matrix"])
    print("bad: %s" % get_counts(M_bad))

In [5]:
m = 7
k = 1
data, best_index, worst_index = get_best_worst(m, k, 'generate_doubly_regular_col')

In [12]:
for info in data:
    print(info["num_errors"])

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [60]:
k = 2
m = 6
data, best_index, worst_index = get_best_worst(m, k, 'generate_doubly_regular_col')

In [42]:
print_inner_product_counts(data, best_index, worst_index)

good: {0.0: 64122, 1.0: 60518, 2.0: 19698, 3.0: 2566, 4.0: 164, 5.0: 4, 6.0: 384}
110592.0
average correlation: 0.3681462140992167
bad: {0.0: 64040, 1.0: 60782, 2.0: 19410, 3.0: 2678, 4.0: 158, 5.0: 4, 6.0: 384}
110592.0
average correlation: 0.3681462140992167


In [70]:
k = 3
m = 4
print_inner_product_counts(*get_best_worst(m, k, 'generate_doubly_regular_col'))

good: {0.0: 103780, 1.0: 39086, 2.0: 4088, 3.0: 118, 4.0: 384}
49152.0
average correlation: 0.1618798955613577
bad: {0.0: 103788, 1.0: 39066, 2.0: 4104, 3.0: 114, 4.0: 384}
49152.0
average correlation: 0.1618798955613577


In [71]:
k = 4
m = 5
print_inner_product_counts(*get_best_worst(m, k, 'generate_doubly_regular_col'))

good: {0.0: 84016, 1.0: 52016, 2.0: 10286, 3.0: 724, 4.0: 30, 5.0: 384}
76800.0
average correlation: 0.2545691906005222
bad: {0.0: 83908, 1.0: 52188, 2.0: 10246, 3.0: 720, 4.0: 10, 5.0: 384}
76800.0
average correlation: 0.2545691906005222


# Block Matrices for column weight = 6

In [9]:
import random
def get_block_matrix(shape = (48, 384), m = 6): 
    """ONLY FOR SHAPE = (48, 384) and m = 6"""
    M = np.zeros((48, 384))
    samples = list(range(384))
    horizontal = int(384 / 8)
    for block in range(0, 6): 
        for i in range(0, 8): 
            M[i + block * 8][samples[horizontal * i:horizontal * (i + 1)]] = 1
        random.shuffle(samples)
    return M

In [14]:
def get_num_errors(k, generate_matrix): 
    block_matrix_dir = COVID_dir + '/tests/results/' + generate_matrix + '/'
    with open(block_matrix_dir + 'm6-k%s-n384-T48-numM25.txt' % k) as file: 
        data = json.load(file)
    num_errors = []
    for info in data:
        num_errors.append(info["num_errors"])
    return num_errors 

In [15]:
for k in range(1, 6): 
    num_errors_block = sum(get_num_errors(k, 'get_block_matrix'))/num_random_matrices
    num_errors_dr = sum(get_num_errors(k, 'generate_doubly_regular_col'))/num_random_matrices
    print("k = %s: \n avg num errors for block = %s \n avg num errors for doubly regular = %s \n " % (k, num_errors_block, num_errors_dr))

k = 1: 
 avg num errors for block = 0.36 
 avg num errors for doubly regular = 0.16 
 
k = 2: 
 avg num errors for block = 5.24 
 avg num errors for doubly regular = 2.64 
 
k = 3: 
 avg num errors for block = 49.12 
 avg num errors for doubly regular = 47.12 
 
k = 4: 
 avg num errors for block = 172.32 
 avg num errors for doubly regular = 150.16 
 
k = 5: 
 avg num errors for block = 298.08 
 avg num errors for doubly regular = 261.28 
 


### Even if we are not optimizing for the column row weight, dr can have better performance than shental

In [18]:
num_errors_dr = []
for k in range(1, 6): 
    num_errors_dr.append(min(get_num_errors(k, 'generate_doubly_regular_col')))

In [19]:
dict(zip(list(range(1, 6)), num_errors_dr))

{1: 0, 2: 0, 3: 30, 4: 120, 5: 227}

In [20]:
shental = {1: 0, 2: 1, 3: 49, 4: 163, 5: 267} 

In [21]:
shental

{1: 0, 2: 1, 3: 49, 4: 163, 5: 267}